In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install ucimlrepo

  Obtaining dependency information for ucimlrepo from https://files.pythonhosted.org/packages/3e/4a/ecc3456479d687202b34ee42317c3a63e09793c9409a720052d38356431a/ucimlrepo-0.0.3-py3-none-any.whl.metadata

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [30]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
breast_cancer_wisconsin_original = fetch_ucirepo(id=15) 
  
# data (as pandas dataframes) 
X = breast_cancer_wisconsin_original.data.features 
y = breast_cancer_wisconsin_original.data.targets 

X.fillna(value=0, inplace=True)

# metadata 
print(breast_cancer_wisconsin_original.metadata) 
  
# variable information 
print(breast_cancer_wisconsin_original.variables) 

{'uci_id': 15, 'name': 'Breast Cancer Wisconsin (Original)', 'repository_url': 'https://archive.ics.uci.edu/dataset/15/breast+cancer+wisconsin+original', 'data_url': 'https://archive.ics.uci.edu/static/public/15/data.csv', 'abstract': 'Original Wisconsin Breast Cancer Database', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 699, 'num_features': 9, 'feature_types': ['Integer'], 'demographics': [], 'target_col': ['Class'], 'index_col': ['Sample_code_number'], 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1990, 'last_updated': 'Fri Nov 03 2023', 'dataset_doi': '10.24432/C5HP4Z', 'creators': ['WIlliam Wolberg'], 'intro_paper': None, 'additional_info': {'summary': "Samples arrive periodically as Dr. Wolberg reports his clinical cases. The database therefore reflects this chronological grouping of the data. This grouping information appears immediately below, having been removed fro

/var/folders/_n/lz98rzln403f9jgxhtvhf_wr0000gn/T/ipykernel_79145/2362441507.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(value=0, inplace=True)


In [31]:
print(X.describe)
print(y.describe)

<bound method NDFrame.describe of      Clump_thickness  Uniformity_of_cell_size  Uniformity_of_cell_shape  \
0                  5                        1                         1   
1                  5                        4                         4   
2                  3                        1                         1   
3                  6                        8                         8   
4                  4                        1                         1   
..               ...                      ...                       ...   
694                3                        1                         1   
695                2                        1                         1   
696                5                       10                        10   
697                4                        8                         6   
698                4                        8                         8   

     Marginal_adhesion  Single_epithelial_cell_size  Bare_nuclei 

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=5)

In [33]:
min_train = X_train.min()
range_train = (X_train - min_train).max()
X_train_scaled = (X_train - min_train)/range_train

min_test = X_test.min()
range_test = (X_test - min_test).max()
X_test_scaled = (X_test - min_test)/range_test

In [41]:
X_train_scaled.head()

,Clump_thickness,Uniformity_of_cell_size,Uniformity_of_cell_shape,Marginal_adhesion,Single_epithelial_cell_size,Bare_nuclei,Bland_chromatin,Normal_nucleoli,Mitoses
129,0.000000,0.000000,0.000000,0.000000,1.000000,0.1,0.000000,0.000000,0.000000
574,1.000000,0.888889,0.666667,0.222222,0.333333,0.2,0.666667,0.666667,0.000000
445,0.111111,0.000000,0.000000,0.000000,0.111111,0.1,0.000000,0.000000,0.000000
488,0.222222,0.555556,0.333333,1.000000,0.222222,0.3,0.222222,0.333333,0.000000
605,1.000000,0.444444,1.000000,0.222222,0.444444,0.8,0.666667,0.777778,0.222222


In [36]:
import numpy as np

from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

svc_model = SVC()
svc_model.fit(X_train_scaled, np.ravel(y_train))
y_predict = svc_model.predict(X_test_scaled)
cm = confusion_matrix(y_test, y_predict)

In [44]:
cm

array([[86,  5],
       [ 2, 47]])

In [39]:
from sklearn.model_selection import GridSearchCV

param_grid = {"C": [0.1, 1, 10, 100], "gamma": [1, 0.1, 0.01, 0.001], "kernel": ["rbf"]}
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=4)
grid.fit(X_train_scaled, np.ravel(y_train))

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.964 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.973 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.964 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.973 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=1.000 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.964 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.964 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.973 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.964 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.964 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.679 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf;

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf']},
             verbose=4)

In [40]:
grid.best_params_

{'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}

In [ ]:
import pandas as pd

In [3]:
output_file = "../data/output/experiment2024-02-06-11-24-28.csv"

data = pd.read_csv(output_file)

In [4]:
data

,Optimizer,objfname,startTime,EndTime,ExecutionTime,Iter1,Iter2,Iter3,Iter4,Iter5,...,Iter91,Iter92,Iter93,Iter94,Iter95,Iter96,Iter97,Iter98,Iter99,Iter100
0,GSA,F1,2024-02-06-11-24-28,2024-02-06-11-24-29,0.54548,21634.17186,13724.250576,5564.349644,1210.276746,439.623279,...,317.524882,317.524882,317.524882,317.524882,317.524882,317.524882,317.524882,317.524882,317.524882,317.524882


In [43]:
# Nº de variables discretas:
dim_d = len(X.columns)

# Nº variables reales (C y Gamma):
dim_r = 2

9

In [ ]:
population_size = 5 
iterations = 20

g_zero_real= 100  # OK
alpha = 20  # OK
g_zero_discrete = 1  # OK

fitness_evaluations = 100

In [47]:
# Filter features
# Set C and Gamma
solution = {'real': [0.1, 0.2], 'discrete': np.array([0, 1, 1, 0, 0, 1, 0, 1, 1])}
C, gamma = solution['real']

X_train_scaled_filtered = X_train_scaled.iloc[:, solution['discrete'] == 1]

print(X_train_scaled.columns)
print(X_train_scaled_filtered.columns)

Index(['Clump_thickness', 'Uniformity_of_cell_size',
       'Uniformity_of_cell_shape', 'Marginal_adhesion',
       'Single_epithelial_cell_size', 'Bare_nuclei', 'Bland_chromatin',
       'Normal_nucleoli', 'Mitoses'],
      dtype='object')
Index(['Uniformity_of_cell_size', 'Uniformity_of_cell_shape', 'Bare_nuclei',
       'Normal_nucleoli', 'Mitoses'],
      dtype='object')


In [51]:
svc_model = SVC(C=C, gamma=gamma, kernel="rbf")
svc_model.fit(X_train_scaled_filtered, np.ravel(y_train))
y_predict = svc_model.predict(X_train_scaled_filtered)
cm = confusion_matrix(y_train, y_predict)

cm

array([[360,   7],
       [ 12, 180]])

In [2]:
import numpy as np

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from typing import Mapping, Tuple, Union
from ucimlrepo import fetch_ucirepo

from benchmarks import gsa_svm_fitness

# fetch dataset 
breast_cancer_wisconsin_original = fetch_ucirepo(id=15) 
  
# data (as pandas dataframes) 
X = breast_cancer_wisconsin_original.data.features 
X = X.fillna(value=0)

y = breast_cancer_wisconsin_original.data.targets 

class UCI:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=5)
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        
        min_train = X_train.min()
        range_train = (X_train - min_train).max()
        self.X_train_scaled = (X_train - min_train)/range_train
        
        min_test = X_test.min()
        range_test = (X_test - min_test).max()
        self.X_test_scaled = (X_test - min_test)/range_test
    
    def get_fitness(self,
                    sol: Mapping[str, np.ndarray],
                    dat: Union[None, Tuple[np.ndarray, np.ndarray]] = None,
                    seed: int = 42
                    ) -> Tuple[float, float]:
        
        if dat is None:
            X_scaled = self.X_train_scaled
            y_dat = self.y_train
        else:
            X_scaled, y_dat = dat
            
        C, gamma = sol['real']
        X_scaled_filtered = X_scaled.iloc[:, sol['discrete'].astype(int) == 1]
        svc_model = SVC(C=C, gamma=gamma, kernel="rbf", verbose=True)
        svc_model.fit(X_scaled_filtered, np.ravel(y_dat))
        y_predict = svc_model.predict(X_scaled_filtered)
        cm = confusion_matrix(y_dat, y_predict)
        
        return gsa_svm_fitness(conf_matrix=cm, solution=sol)

uci = UCI(X, y)

In [55]:
solution = {'real': [5, 5], 'discrete': np.array([0, 0, 0, 0, 0, 0, 0, 0, 1])}
uci.get_fitness(solution=solution)

62.71802822500498

In [6]:
import csv
import numpy as np
import os
import time

from benchmarks import get_function_details
from src.entities import GSA

# Select number of repetitions for each experiment. 
# To obtain meaningful statistical results, usually 30 independent runs 
# are executed for each algorithm.
runs = 10

# Select chaotic constant
chaotic_constant = True

# Export results ?
export = True

save_path = "data/output/"
if not os.path.exists(save_path):
    os.makedirs(save_path)

ExportToFile = save_path + "experiment" + time.strftime("%Y-%m-%d-%H-%M-%S_") + str(chaotic_constant) + ".csv"

# Check if it works at least once
atLeastOneIteration = False

population_size = 5 
iterations = 20

fitness_evaluations = 100

boundaries = {'real': [(0.0001, 100), (0.0001, 100)], 'discrete': [(0, 1) for _ in range(len(X.columns))]}

# CSV Header for the convergence
CnvgHeader = ["Sol_Iter"+str(l+1) for l in range(iterations)]

best_solution_history = []
accuracy_history = []
gamma_history = []
c_history = []
n_features_history = []

uci = UCI(X, y)

for k in range(runs):
    gsa_algo = GSA(objective_function = uci.get_fitness,
                   r_dim=2,
                   d_dim=len(X.columns),
                   boundaries=boundaries)
    
    gsa_algo.optimize(population_size=population_size,
                      iters=iterations,
                      chaotic_constant=True)
    
    print(gsa_algo.solution_history[-1])
    fitness, accuracy = uci.get_fitness(sol=gsa_algo.solution_history[-1],
                                        dat=(uci.X_test_scaled, uci.y_test))
    
    print("Test accuracy: ", accuracy, " - Fitness: ", fitness)
    
    best_solution_history.append(gsa_algo.solution_history[-1])
    accuracy_history.append(accuracy)
    gamma_history.append(gsa_algo.solution_history[-1]['real'][1])
    c_history.append(gsa_algo.solution_history[-1]['real'][0])
    n_features_history.append(np.sum(gsa_algo.solution_history[-1]['discrete']))
    
    if export:
        with open(ExportToFile, 'a') as out:
            writer = csv.writer(out, delimiter=',')
            if not atLeastOneIteration:  # just one time to write the header of the CSV file
                header = np.concatenate(
                    [["Optimizer", "objfname", "startTime", "EndTime", "ExecutionTime"], CnvgHeader])
                writer.writerow(header)
            a = np.concatenate(
                [[gsa_algo.objective_function_name,
                  gsa_algo.start_time,
                  gsa_algo.end_time,
                  gsa_algo.execution_time],
                  gsa_algo.convergence])
            writer.writerow(a)
        out.close()
    atLeastOneIteration = True  # at least one experiment

if not atLeastOneIteration:  # Faild to run at least one experiment
    print("No Optimizer or Cost function is selected. Check lists of available optimizers and cost functions")

GSA is optimizing  "get_fitness"
[LibSVM]*
optimization finished, #iter = 192
obj = -15.625012, rho = -0.971405
nSV = 384, nBSV = 384
Total nSV = 384
[LibSVM]*
optimization finished, #iter = 225
obj = -547.370054, rho = -0.626944
nSV = 51, nBSV = 37
Total nSV = 51
[LibSVM]*
optimization finished, #iter = 58
obj = -431.211118, rho = 0.086988
nSV = 49, nBSV = 43
Total nSV = 49
[LibSVM]*
optimization finished, #iter = 146
obj = -12.357577, rho = 0.083142
nSV = 290, nBSV = 288
Total nSV = 290
[LibSVM]*
optimization finished, #iter = 269
obj = -54.475000, rho = 0.691836
nSV = 116, nBSV = 46
Total nSV = 116
['At iteration 1 the best fitness is 52.655694692904']
[LibSVM]*
optimization finished, #iter = 261
obj = -73.647477, rho = 0.754600
nSV = 122, nBSV = 24
Total nSV = 122
[LibSVM]*
optimization finished, #iter = 192
obj = -0.038278, rho = -0.987928
nSV = 384, nBSV = 384
Total nSV = 384
[LibSVM]*
optimization finished, #iter = 192
obj = -0.038264, rho = -0.986701
nSV = 384, nBSV = 384
Total

In [7]:
best_solution_history

[{'real': array([0.0001, 0.0001]),
  'discrete': array([1, 1, 1, 1, 1, 1, 1, 1, 1])},
 {'real': array([1.00000000e-04, 2.70494315e+01]),
  'discrete': array([1, 1, 1, 1, 1, 1, 1, 0, 0])},
 {'real': array([0.0001, 0.0001]),
  'discrete': array([1, 1, 1, 1, 1, 1, 1, 1, 1])},
 {'real': array([0.0001, 0.0001]),
  'discrete': array([1, 1, 1, 1, 1, 1, 1, 1, 1])},
 {'real': array([1.00000000e-04, 5.91431431e+00]),
  'discrete': array([1, 1, 0, 1, 1, 1, 1, 1, 1])},
 {'real': array([1.0000000e-04, 2.5379953e-01]),
  'discrete': array([1, 1, 1, 1, 1, 1, 1, 1, 1])},
 {'real': array([0.0001, 0.0001]),
  'discrete': array([1, 1, 1, 1, 1, 1, 1, 1, 1])},
 {'real': array([5.2717133e+00, 1.0000000e-04]),
  'discrete': array([1, 1, 1, 1, 1, 1, 1, 1, 1])},
 {'real': array([0.0001, 0.0001]),
  'discrete': array([1, 1, 1, 1, 1, 1, 1, 1, 1])},
 {'real': array([4.27840216e-01, 1.00000000e-04]),
  'discrete': array([1, 1, 1, 0, 1, 1, 1, 1, 1])}]

In [4]:
accuracy_history

[65.0, 65.0, 65.0, 65.0, 65.0, 65.0, 65.0, 65.0, 65.0, 65.0]

In [5]:
mean_accuracy = np.mean(accuracy_history)
std_accuracy = np.std(accuracy_history)

print(f"Mean accuracy: {mean_accuracy} +/- {std_accuracy}")

mean_gamma = np.mean(gamma_history)
std_gamma = np.std(gamma_history)

print(f"Mean gamma: {mean_gamma} +/- {std_gamma}")

mean_c = np.mean(c_history)
std_c = np.std(c_history)

print(f"Mean C: {mean_c} +/- {std_c}")

mean_n_features = np.mean(n_features_history)
std_n_features = np.std(n_features_history)

print(f"Mean n_features: {mean_n_features} +/- {std_n_features}")

Mean accuracy: 65.0 +/- 0.0
Mean gamma: 13.162493244930562 +/- 25.70906214660191
Mean C: 0.4198881484336175 +/- 1.2593644453008526
Mean n_features: 8.6 +/- 0.6633249580710799


In [59]:
gsa_algo.solution_history

[{'real': array([ 4.47868902, 17.23375128]),
  'discrete': array([0., 1., 0., 0., 0., 1., 1., 0., 0.])},
 {'real': array([ 4.47868902, 17.23375128]),
  'discrete': array([0., 1., 0., 0., 0., 1., 1., 0., 0.])},
 {'real': array([26.98106586,  4.99250484]),
  'discrete': array([1., 0., 0., 1., 1., 0., 0., 0., 0.])},
 {'real': array([28.61304963,  1.86654023]),
  'discrete': array([1., 0., 0., 1., 1., 0., 0., 0., 0.])},
 {'real': array([23.14743053,  0.        ]),
  'discrete': array([1., 1., 0., 0., 0., 1., 1., 0., 0.])},
 {'real': array([23.14743053,  0.        ]),
  'discrete': array([1., 1., 0., 0., 0., 1., 1., 0., 0.])},
 {'real': array([23.14743053,  0.        ]),
  'discrete': array([1., 1., 0., 0., 0., 1., 1., 0., 0.])},
 {'real': array([23.14743053,  0.        ]),
  'discrete': array([1., 1., 0., 0., 0., 1., 1., 0., 0.])},
 {'real': array([23.14743053,  0.        ]),
  'discrete': array([1., 1., 0., 0., 0., 1., 1., 0., 0.])},
 {'real': array([23.14743053,  0.        ]),
  'discret

In [5]:
gsa_algo.convergence

array([52.61125025, 52.61125025, 52.61125025, 52.61125025, 52.61125025,
       52.61125025, 52.61125025, 52.61125025, 52.61125025, 52.61125025,
       52.61125025, 52.61125025, 52.61125025, 52.61125025, 52.61125025,
       52.61125025, 52.61125025, 52.61125025, 52.61125025, 52.61125025])

In [67]:
[10**(-1 * np.random.uniform(0.01, 0.1)) for _ in range(10)]

[0.9148689453320515,
 0.8713689649925521,
 0.9588229965319557,
 0.9312029250189291,
 0.9608240254783925,
 0.8861057333462302,
 0.8191249516341255,
 0.8681967724478741,
 0.8938163600540469,
 0.9161701467210402]

In [71]:
numeros_lineales = np.random.uniform(low=np.log10(0.0001), high=np.log10(100), size=10)
numeros_logaritmicos = 10**numeros_lineales
numeros_logaritmicos

array([1.48722118e-01, 9.03596168e-03, 2.11233000e+01, 9.75965664e-01,
       1.12213676e-04, 5.30770215e+00, 1.40524427e+00, 2.39949961e-01,
       2.57473813e-04, 4.46506892e+01])

In [50]:
solution= {'real': np.array([0.00193985, 0.35698041]),
           'discrete': np.array([1, 0, 1, 1, 0, 1, 1, 0, 1])}

fitness, accuracy = uci.get_fitness(sol=solution,
                                    dat=(uci.X_test_scaled, uci.y_test))

print("Test accuracy: ", accuracy, " - Fitness: ", fitness)

Test accuracy:  65.0  - Fitness:  52.06666666666667


In [15]:
# Ejemplo de uso
# Supongamos que tienes dos arrays arr1 y arr2
arr1 = np.array([0, 1, 0, 1, 1])
arr2 = np.array([0.2, 0.8, 0.5, 0.3, 0.9])

rand = np.random.rand(*arr1.shape)

# Encuentra las posiciones donde rand < arr2 y modifica arr1 en esas posiciones
arr1[rand < arr2] = 1 - arr1[rand < arr2]

arr1

array([0, 0, 0, 1, 0])